## Creación y entrenamiento de una red neuronal con Tensorflow


![tensorflow](../images/tensorflow.jpeg)

### Primero un poco de teoría

![node](../images/node.png)


![neural](../images/neural.jpg)

### Convolucionales

https://en.wikipedia.org/wiki/Convolution


![neural2](../images/neural2.webp)

![neural3](../images/neural3.gif)

![neural4](../images/neural4.png)

# Entrenamiento del modelo (ajuste)

In [ ]:
# librerias
import warnings
warnings.filterwarnings('ignore')

import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import TensorBoard

from imutils import paths
import visualkeras
from IPython.display import Image
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report as report
import datetime

In [ ]:
tema='mascara'

In [ ]:
# visualizacion del entrenamiento con tensorboard, guarda el log

%rm -rf ./logs/
%load_ext tensorboard

**Cargando y preparando datos**

In [ ]:
imagenes=list(paths.list_images(f'../data/dataset_{tema}'))

In [ ]:
imagenes[0]

In [ ]:
Image(url=imagenes[0], width=300)

In [ ]:
# preparando datos

data=[]
etiquetas=[]

for img in imagenes:
    
    # etiqueta segun nombre del archivo
    if 'con_' in img:
        etiquetas.append([0])  # con tema
    else:
        etiquetas.append([1])  # sin tema
    
    
    # carga la imagen (224x224) y procesala
    imagen=load_img(img, target_size=(224, 224))
    imagen=img_to_array(imagen)
    imagen=preprocess_input(imagen)

    # añade a la lista
    data.append(imagen)
    

    
# se convierten datos y etiquetas a arrays de numpy
data=np.array(data, dtype='float32')
etiquetas=np.array(etiquetas)

In [ ]:
data[0].shape

In [ ]:
etiquetas[0]

In [ ]:
# one hot para las etiquetas

etiquetas=to_categorical(etiquetas)

etiquetas[0]

In [ ]:
# train-test split (separa datos de entrenamiento y testeo)

X_train, X_test, y_train, y_test=tts(data, etiquetas, 
                                     test_size=.2, 
                                     stratify=etiquetas, 
                                     random_state=42)

In [ ]:
X_train.shape

In [ ]:
# aumenta datos con el generador de imagenes

img_gen=ImageDataGenerator(rotation_range=20, 
                           zoom_range=.15,
                           width_shift_range=.2, 
                           height_shift_range=.2,
                           shear_range=.15, 
                           horizontal_flip=True,
                           fill_mode='nearest')

## Modelo - Red Neuronal

Se creara el modelo con MobileNetV2, una red para reconocimiento de imagenes preentrenada.

**constantes**

In [ ]:
TASA=1e-4    # tasa de aprendizaje

EPOCAS=5     # epocas de entrenamiento

BATCH=32     # tamaño del batch (muestra de datos que pasan a la red para entrenar)

**Red**

In [ ]:
# se carga MobileNetV2 sin el clasificador
modelo_base=MobileNetV2(weights='imagenet', 
                        include_top=False, 
                        input_tensor=Input(shape=(224, 224, 3)))


# se contruye el nuevo clasificador
modelo_x=modelo_base.output

modelo_x=AveragePooling2D(pool_size=(7, 7))(modelo_x)

modelo_x=Flatten(name='flatten')(modelo_x)

modelo_x=Dense(128, activation='relu')(modelo_x)

modelo_x=Dropout(.5)(modelo_x)

modelo_x=Dense(2, activation='softmax')(modelo_x)



# se pone todo junto
modelo=Model(inputs=modelo_base.input, outputs=modelo_x)

In [ ]:
# se congela el entrenamiento del modelo base, aprovechamos el preentreno

for capa in modelo_base.layers:
    capa.trainable=False

In [ ]:
# se compila el modelo

optimizador=Adam(lr=TASA, 
                 decay=TASA / EPOCAS)

modelo.compile(loss='binary_crossentropy', 
               optimizer=optimizador, 
               metrics=['accuracy'])

In [ ]:
modelo.summary() # resumen de la red

In [ ]:
visualkeras.layered_view(modelo)

In [ ]:
%tensorboard --logdir logs/fit

http://localhost:6006/

In [ ]:
#help(TensorBoard)

In [ ]:
%%time

# se entrena el clasificador con las imagenes

print('Entrenando modelo...')

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, 
                                   histogram_freq=1,
                                   write_graph=1,
                                   write_images=1,
                                   write_steps_per_second=1)


historico=modelo.fit(img_gen.flow(X_train, y_train, batch_size=BATCH), 
                     steps_per_epoch=len(X_train)//BATCH,
                     validation_data=(X_test, y_test), 
                     validation_steps=len(X_test)//BATCH, 
                     epochs=EPOCAS,
                     callbacks=[tensorboard_callback])

print('Entrenado.')

In [ ]:
# haciendo predicciones
preds=modelo.predict(X_test, batch_size=BATCH)

preds[0]  # en probabilidad

In [ ]:
# la mayor prob de pertenencia, en 0/1

preds=np.argmax(preds, axis=1)

preds[0]  

In [ ]:
# informe de clasificacion

print(report(y_test.argmax(axis=1), preds, target_names=['con_', 'sin_']))

In [ ]:
# guardando modelo en formato h5

modelo.save(f'../models/{tema}.model', save_format='h5')
